# About

Code snippet for finding closest image dimensions for which cuFFT uses faster Cooley-Tukey implementation (as opposed to Bluestein).
See https://docs.nvidia.com/cuda/cufft/index.html

Quote from there:
_Algorithms highly optimized for input sizes that can be written in the form 2 a × 3 b × 5 c × 7 d . In general the smaller the prime factor, the better the performance, i.e., powers of two are fastest._ 


Volker dot Hilsenstein at monash dot edu, April 2019

# Requirements


This notebook requires the following additional packages. Use `pip` or `conda` to install them.

* `sympy`
* `numpy`
* `tqdm`

#  Implementation

In [1]:
from sympy.ntheory import factorint
import warnings
import numpy as np

def is_optimal_for_cuFFT(n: int, allowed_factors) -> bool:
    factorization = factorint(n)
    if len(factorization) == 0: # factorint(1) returns empyt dict
        return False
    factors = set(factorization.keys())
    return factors.issubset(set(allowed_factors))
    
def _closest_optimal(n: int, search_next_largest: bool, allowed_factors) -> int:
    while(not is_optimal_for_cuFFT(n, allowed_factors) and n>=1):
        if search_next_largest:
            n += 1
        else:
            n -= 1
    # edge case: decreasing search with start value smaller than allowed factor
    if n < min(allowed_factors):
        
        warnings.warn(f"{n}One provided dimension is smaller than smallest allowed factor and search direction is decreasing")
        return(min(allowed_factors))
    return n

def closest_optimal(n, search_next_largest: bool=True, allowed_factors=(2,3,5,7)):
    """ Finds closest optimal array dimensions for cuFFT
    
    Parameters
    ----------
    n : iterable of integers
        Input dimensions
    search_next_largest : bool
        if True (default) search closest optimal dimensions that are larger or equal to original
        otherwise look for smaller ones. 
    allowed_factor: tuple of integers
        allowed factors in decomposition. Defaults to (2,3,5,7) which are the factors listed in 
        the cuFFT documentation. 
    
    Returns
    -------
    np.array of ints
        optimal dimensions for cuFFT
        
        
    See also
    --------
    https://docs.nvidia.com/cuda/cufft/index.html
    
    """
    n = np.asarray(n)
    scalar_input = False
    if n.ndim == 0:
        n = n[None] 
        scalar_input = True
    ret = np.array([_closest_optimal(ni, search_next_largest, allowed_factors) for ni in n])
    if scalar_input:
        return ret[0]
    return ret

# Examples

In [2]:
# Simple case, single number
closest_optimal(123)

125

In [3]:
# find a smaller optimal dimension
closest_optimal(123, search_next_largest=False)

120

In [4]:
# don't allow all factors
closest_optimal(123, search_next_largest=False, allowed_factors=(2,3))

108

In [5]:
# only allow a single factor
# use a comma to make it a tuple, otherwise it will throw an error!
closest_optimal(123, search_next_largest=False, allowed_factors=(2,))

64

In [6]:
# apply to multiple dimensions
closest_optimal((123, 23, 615))

array([125,  24, 625])

In [7]:
# edge case, one dimension smaller than smallest factor and decreasing search should generate a warning
closest_optimal((1, 23, 615), search_next_largest=False)

c:\users\volker\anaconda3\envs\spimenv\lib\site-packages\ipykernel_launcher.py:21: UserWarning: 0One provided dimension is smaller than smallest allowed factor and search direction is decreasing


array([  2,  21, 600])

In [8]:
# one dimension smaller than smallest factor and increasing search should not generate a warning
closest_optimal((1, 23, 615))

array([  2,  24, 625])

# Todo

* could allow `search_next_largest` to be an iterable of bools, to apply different strategies (rounding up/rounding down) according to dimension.
* could remove `sympy`-dependency by implementing recursive modulo tests as in `notGoodDimension` from https://github.com/dmilkie/cudaDecon/blob/master/RL-Biggs-Andrews.cpp. However, I find the explicit factorization more readable than the recursion.

# Rainbow table

Create a rainbow table with pre-computed good dimensions. Note that with this idiotic approach of building a rainbow table is very slow (building by multiplication is quicker than building by factorization). But we only have to do it once.

## idiotic approach (building by factorization)

Be lazy, recycle the code defined above and start testing numbers.
The tqdm progress bar indicates the problem.
Big-O for factorization is bad, hence used for encryption.

In [9]:
import tqdm
def create_cuFFT_dim_rainbow_table(nr_of_els: int = 1000):
    good_dimensions = [0] * nr_of_els
    val = 1
    for i in tqdm.tqdm(range(nr_of_els)):
        good_dimensions[i] = closest_optimal(val)
        val = good_dimensions[i]+1
    return good_dimensions

In [10]:
good_dimensions = create_cuFFT_dim_rainbow_table()

100%|███████████████████████████████████████████████| 1000/1000 [00:06<00:00, 155.06it/s]


In [11]:
print(repr(good_dimensions))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 50, 54, 56, 60, 63, 64, 70, 72, 75, 80, 81, 84, 90, 96, 98, 100, 105, 108, 112, 120, 125, 126, 128, 135, 140, 144, 147, 150, 160, 162, 168, 175, 180, 189, 192, 196, 200, 210, 216, 224, 225, 240, 243, 245, 250, 252, 256, 270, 280, 288, 294, 300, 315, 320, 324, 336, 343, 350, 360, 375, 378, 384, 392, 400, 405, 420, 432, 441, 448, 450, 480, 486, 490, 500, 504, 512, 525, 540, 560, 567, 576, 588, 600, 625, 630, 640, 648, 672, 675, 686, 700, 720, 729, 735, 750, 756, 768, 784, 800, 810, 840, 864, 875, 882, 896, 900, 945, 960, 972, 980, 1000, 1008, 1024, 1029, 1050, 1080, 1120, 1125, 1134, 1152, 1176, 1200, 1215, 1225, 1250, 1260, 1280, 1296, 1323, 1344, 1350, 1372, 1400, 1440, 1458, 1470, 1500, 1512, 1536, 1568, 1575, 1600, 1620, 1680, 1701, 1715, 1728, 1750, 1764, 1792, 1800, 1875, 1890, 1920, 1944, 1960, 2000, 2016, 2025, 2048, 2058, 2100, 2160, 2187, 2205, 2240, 2250, 2268, 2304, 23

In [12]:
good_dimensions_fac = good_dimensions # save for later

## sensible approach (building by multiplication)

In [13]:
# increase these maximum exponents as desired
n7 = 36
n5 = 43
n3 = 63
n2 = 100

In [14]:
max_to_consider = min(2**n2, 3**n3, 5**n5, 7**n7) 
i = 0
good_dimensions = []
for pow7 in tqdm.tqdm(range(n7+1)):
    for pow5 in range(n5+1):
        for pow3 in range(n3+1):
            for pow2 in range(n2+1):
                prod = 2**pow2 * 3**pow3 * 5**pow5 * 7**pow7
                if prod < max_to_consider:
                    good_dimensions.append(prod)
print("sorting")
good_dimensions = sorted(good_dimensions)
# drop the 1
good_dimensions = good_dimensions[1:]

100%|████████████████████████████████████████████████████| 37/37 [00:20<00:00,  1.73it/s]


sorting


In [15]:
# Check both the factorization and the multiplication approach give the same first 1000 elements
assert(good_dimensions_fac[:] == good_dimensions[:1000])

# Rainbow table lookup

In [16]:
from bisect import bisect_left
lookup_larger = lambda x : good_dimensions[bisect_left(good_dimensions, x)]
lookup_smaller = lambda x : good_dimensions[bisect_left(good_dimensions, x)-1]

In [17]:
testnr = 11223344

print("Rainbow lookup:")
print("next largest :", lookup_larger(testnr))
print("nex smallest :", lookup_smaller(testnr))

print("Factorization check:")
print("next largest :", closest_optimal(testnr))
print("nex smallest :", closest_optimal(testnr, search_next_largest=False ))

Rainbow lookup:
next largest : 11239424
nex smallest : 11200000
Factorization check:
next largest : 11239424
nex smallest : 11200000


# Timing

### factorization

In [18]:
%%timeit
closest_optimal(testnr)

939 ms ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### rainbow table

In [19]:
%%timeit 
lookup_larger(testnr)

617 ns ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# Serialize rainbow table into usable python code

Generate a `.py` file with the first `n_elements` of the rainbow table.

Note that the lambda functions don't deal with any edge cases and work on scalars only.
They will throw an exception if the input value exceeds the largest value in the rainbow table. 

In [20]:
n_elements = 8000

py_code = """ 
from bisect import bisect_left

good_dimensions = {rainbowtable}

lookup_larger = lambda x : good_dimensions[bisect_left(good_dimensions, x)]
lookup_smaller = lambda x : good_dimensions[bisect_left(good_dimensions, x)-1]
""".format(rainbowtable=repr(good_dimensions[:n_elements]))

with open("rainbow_cufft_pad.py", "w") as pyfile:
    pyfile.write(py_code)